# Data matching
---

Experimenting with matching data from:
- Global Energy Monitor (GEM)'s [Global Coal Plant Tracker](https://www.globalenergymonitor.org/coal.html)
- USA's [CAMPD emissions data](https://campd.epa.gov/data)
- OSM's [cooling_tower](https://wiki.openstreetmap.org/wiki/Tag:man_made%3Dcooling_tower) tag

## Setup

### Imports

In [ ]:
import requests
import overpy
import pandas as pd
import geopandas as gpd
from pathlib import Path
from typing import Union

### APIs

In [ ]:
osm_api = overpy.Overpass()

### Parameters

In [ ]:
start_date = "2022-01-01"
end_date = "2022-12-31"

In [ ]:
# show all columns in pandas
pd.set_option("display.max_columns", None)

### Functions

In [ ]:
def load_raw_gcpt_data(gcpt_path: Union[str, Path]) -> pd.DataFrame:
    """
    Load GCPT data in its raw excel format from GCS.

    Returns:
        df (pd.DataFrame):
            GCPT data frame
    """
    df = pd.read_excel(
        gcpt_path,
        sheet_name="Units",
    )
    return df


def clean_gcpt_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean the GCPT data frame, setting better column names.

    Args:
        df (pd.DataFrame):
            GCPT data frame

    Returns:
        df (pd.DataFrame):
            Cleaned GCPT data frame
    """
    df.columns = (
        df.columns.str.lower()
        .str.replace(" ", "_")
        .str.replace("(", "")
        .str.replace(")", "")
        .str.replace("/", "_")
        .str.replace("-", "_")
        .str.replace(",", "_")
    )
    df.rename(columns={"parentid": "parent_id"}, inplace=True)
    df.rename(columns={"trackerloc": "tracker_loc"}, inplace=True)
    return df

## Load data

### GEM Global Coal Plant Tracker

In [ ]:
gcpt_df = clean_gcpt_dataframe(
    load_raw_gcpt_data("/Users/adminuser/Downloads/Global-Coal-Plant-Tracker-January-2023.xlsx")
)
gcpt_df

In [ ]:
# convert to geodataframe
gcpt_gdf = gpd.GeoDataFrame(
    gcpt_df,
    geometry=gpd.points_from_xy(gcpt_df.longitude, gcpt_df.latitude),
    crs="EPSG:4326",
)
gcpt_gdf

### CAMPD emissions data

### OSM cooling_tower tag

In [ ]:
osm_results = osm_api.query(
    """
    [out:json];
    // fetch area "United States of America" to use as a boundary
    {{geocodeArea:United States of America}}->.searchArea;

    // query for nodes, ways, and relations with man_made=cooling_tower tag within the search area
    (
    node["man_made"="cooling_tower"](area.searchArea);
    way["man_made"="cooling_tower"](area.searchArea);
    relation["man_made"="cooling_tower"](area.searchArea);
    );

    // output the results
    out body;
    >;
    out skel qt;
    """
)
osm_results

In [ ]:
# convert OSM results to geodataframe
osm_gdf = gpd.GeoDataFrame.from_features(osm_results.features)
osm_gdf

## Match data